In [16]:
import google.auth
import google.auth.transport.requests
import requests
import json
from google.cloud import aiplatform
creds, project = google.auth.default()

# creds.valid is False, and creds.token is None
# Need to refresh credentials to populate those

auth_req = google.auth.transport.requests.Request()
creds.refresh(auth_req)
token = creds.token

aiplatform.init(
    project='vidio-quiz-prod',
    location='asia-southeast1',
    staging_bucket='gs://genai_hackathon_2024',
)

model = aiplatform.Endpoint("7738653107357220864")

def embedding_text(model, text):
    prediction = model.predict(instances=[{
        "content": text,
        "task_type": "DEFAULT",
        "title": ""
    }])
    for embedding in prediction.predictions:
        vector = embedding
    return vector

data_store = "film-metadata-202403191330_1710829784824"
project_id = "328583281153"
location = "global"

In [54]:
def vertex_search_http(query):
    url = f"https://discoveryengine.googleapis.com/v1alpha/projects/{project_id}/locations/{location}/collections/default_collection/dataStores/{data_store}/servingConfigs/default_search:search"
    data = {
        "query": query, 
        "pageSize": 10,
        # "embeddingSpec": {
        #     "embeddingVectors": [{
        #     "fieldPath": "embedding_vector",
        #     "vector": embedding_text(model, query)
        #     }]
        # },
        # "ranking_expression": "1.0 * relevance_score + 0.0 * dotProduct(embedding_vector)"
    }
    headers = {"Authorization": f"Bearer {token}"}
    response = requests.post(url, json=data, headers=headers)
    response_json = json.loads(response.text)
    result = response_json
    result = list(map(lambda x: x["document"]["structData"], response_json["results"]))
    result = list(map(lambda x: {"title": x['title'], "actors": x['actors']}, result))

    return result

In [55]:
vertex_search_http('reza rahadian')

[{'title': 'cinta badut cinta', 'actors': ['reza rahadian', 'sylvia fully']},
 {'title': 'terpana',
  'actors': ['fachri albar', 'raline shah', 'reza rahadian']},
 {'title': 'jatuh cinta beneran', 'actors': ['aulia sarah', 'reza rahadian']},
 {'title': 'the gift',
  'actors': ['ayushita', 'christine hakim', 'dion wiyoko', 'reza rahadian']},
 {'title': 'firegate',
  'actors': ['julie estelle', 'ray sahetapy', 'reza rahadian']},
 {'title': 'saat kucing dan anjing jatuh cinta',
  'actors': ['reza rahadian', 'sharena']},
 {'title': 'getar 2 cinta', 'actors': ['kimberly ryder', 'reza rahadian']},
 {'title': 'benyamin biang kerok',
  'actors': ['aci resti',
   'adjis doaibu',
   'delia husein',
   'meriam bellina',
   'rano karno',
   'reza rahadian']},
 {'title': 'kapan kawin?',
  'actors': ['adinia wirasti',
   'erwin cortez',
   'feby febiola',
   'ivanka suwandi',
   'reza rahadian']},
 {'title': 'tenggelamnya kapal van der wijck',
  'actors': ['gesya shandy',
   'herjunot ali',
   'jaja